# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [3]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [4]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [5]:
# First things first, let's change the user ids to look more clean.

def id_mapper(column):
    coded_dict = dict()
    counter = 1
    id_encoded = []
    
    for val in column:
        if val not in coded_dict:
            coded_dict[val] = counter
            counter+=1
        
        id_encoded.append(coded_dict[val])
    return id_encoded, coded_dict

In [6]:
user_id_encoded, user_coded_dict = id_mapper(profile['id'])

In [7]:
profile['user_id'] = user_id_encoded

In [8]:
del profile['id']

In [9]:
len(user_id_encoded)

17000

In [10]:
offer_id_encoded, offer_coded_dict = id_mapper(portfolio['id'])

In [11]:
for val in transcript['value']:
    if 'offer id' in val:
        val['offer id'] = offer_coded_dict.get(val['offer id'])
    else:
        pass

In [12]:
transcript

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': 4}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': 5}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': 10}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 7}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': 2}
5,offer received,389bc3fa690240e798340f5a15918d5c,0,{'offer id': 9}
6,offer received,c4863c7985cf408faee930f111475da3,0,{'offer id': 6}
7,offer received,2eeac8d8feae4a8cad5a6af0499a211d,0,{'offer id': 3}
8,offer received,aa4862eba776480b8bb9c68455b8c2e1,0,{'offer id': 5}
9,offer received,31dda685af34476cad5bc968bdb01c53,0,{'offer id': 5}


In [13]:
for val in transcript['value']:
    if 'offer_id' in val:
        val['offer_id'] = offer_coded_dict.get(val['offer_id'])
    else:
        pass

In [14]:
def code_to_id(val):
    val = user_coded_dict.get(val)
    return val

In [15]:
transcript['person'] = transcript['person'].apply(lambda x: code_to_id(x))

In [16]:
transcript

,event,person,time,value
0,offer received,4,0,{'offer id': 4}
1,offer received,5,0,{'offer id': 5}
2,offer received,6,0,{'offer id': 10}
3,offer received,7,0,{'offer id': 7}
4,offer received,8,0,{'offer id': 2}
5,offer received,9,0,{'offer id': 9}
6,offer received,11,0,{'offer id': 6}
7,offer received,13,0,{'offer id': 3}
8,offer received,14,0,{'offer id': 5}
9,offer received,16,0,{'offer id': 5}


In [17]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [18]:
transcript.groupby('person')['event'].count()

person
1        21
2         6
3        10
4        18
5        11
6        12
7        10
8        11
9        20
10       14
11       16
12       17
13       10
14       19
15       19
16       23
17       15
18       15
19       14
20       12
21       30
22       19
23       21
24       15
25       16
26       19
27       26
28       19
29        6
30       26
         ..
16971    16
16972    18
16973    16
16974    33
16975    20
16976     9
16977    19
16978    15
16979    23
16980    33
16981    16
16982    12
16983    29
16984    15
16985    19
16986    13
16987    13
16988    20
16989    20
16990    12
16991    29
16992    10
16993    13
16994     9
16995    16
16996    13
16997    12
16998    12
16999    23
17000    14
Name: event, Length: 17000, dtype: int64

In [19]:
del profile['became_member_on']

In [20]:
transcript

,event,person,time,value
0,offer received,4,0,{'offer id': 4}
1,offer received,5,0,{'offer id': 5}
2,offer received,6,0,{'offer id': 10}
3,offer received,7,0,{'offer id': 7}
4,offer received,8,0,{'offer id': 2}
5,offer received,9,0,{'offer id': 9}
6,offer received,11,0,{'offer id': 6}
7,offer received,13,0,{'offer id': 3}
8,offer received,14,0,{'offer id': 5}
9,offer received,16,0,{'offer id': 5}


In [21]:
transcript.rename(columns={'person': 'user_id'}, inplace=True)

In [22]:
df = transcript.merge(profile, on='user_id', how='inner')

In [23]:
df.head()

,event,user_id,time,value,age,gender,income
0,offer received,4,0,{'offer id': 4},75,F,100000.0
1,offer viewed,4,6,{'offer id': 4},75,F,100000.0
2,transaction,4,132,{'amount': 19.89},75,F,100000.0
3,offer completed,4,132,"{'offer_id': 4, 'reward': 5}",75,F,100000.0
4,transaction,4,144,{'amount': 17.78},75,F,100000.0


In [24]:
for val in df['value']:
    if next(iter(val)) == 'offer_id':
        val['offer id'] = val.pop('offer_id')

In [25]:
df['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [26]:
df_viewed = df[df['event'] == 'offer viewed']

In [27]:
df_viewed

,event,user_id,time,value,age,gender,income
1,offer viewed,4,6,{'offer id': 4},75,F,100000.0
6,offer viewed,4,216,{'offer id': 8},75,F,100000.0
11,offer viewed,4,408,{'offer id': 1},75,F,100000.0
17,offer viewed,4,582,{'offer id': 9},75,F,100000.0
19,offer viewed,5,6,{'offer id': 5},118,None,NaN
23,offer viewed,5,336,{'offer id': 3},118,None,NaN
28,offer viewed,5,624,{'offer id': 5},118,None,NaN
30,offer viewed,6,18,{'offer id': 10},68,M,70000.0
35,offer viewed,6,420,{'offer id': 4},68,M,70000.0
37,offer viewed,6,522,{'offer id': 7},68,M,70000.0


In [28]:
df_completed = df[df['event'] == 'offer completed']

In [29]:
df_completed

,event,user_id,time,value,age,gender,income
3,offer completed,4,132,"{'reward': 5, 'offer id': 4}",75,F,100000.0
14,offer completed,4,510,"{'reward': 10, 'offer id': 1}",75,F,100000.0
15,offer completed,4,510,"{'reward': 5, 'offer id': 9}",75,F,100000.0
39,offer completed,6,522,"{'reward': 5, 'offer id': 4}",68,M,70000.0
40,offer completed,6,522,"{'reward': 2, 'offer id': 7}",68,M,70000.0
65,offer completed,9,60,"{'reward': 5, 'offer id': 9}",65,M,53000.0
73,offer completed,9,498,"{'reward': 2, 'offer id': 10}",65,M,53000.0
74,offer completed,9,498,"{'reward': 5, 'offer id': 4}",65,M,53000.0
80,offer completed,9,600,"{'reward': 5, 'offer id': 9}",65,M,53000.0
81,offer completed,9,600,"{'reward': 2, 'offer id': 7}",65,M,53000.0


In [33]:
# I will create a column for the offer_id_viewed since it is now inside a dictionary

offer_id_viewed = []

for val in df_viewed['value']:
    offer_id_viewed.append(val['offer id'])

In [36]:
df_viewed['offer_id_viewed'] = offer_id_viewed

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
del df_viewed['value']

In [41]:
df_completed

,event,user_id,time,value,age,gender,income
3,offer completed,4,132,"{'reward': 5, 'offer id': 4}",75,F,100000.0
14,offer completed,4,510,"{'reward': 10, 'offer id': 1}",75,F,100000.0
15,offer completed,4,510,"{'reward': 5, 'offer id': 9}",75,F,100000.0
39,offer completed,6,522,"{'reward': 5, 'offer id': 4}",68,M,70000.0
40,offer completed,6,522,"{'reward': 2, 'offer id': 7}",68,M,70000.0
65,offer completed,9,60,"{'reward': 5, 'offer id': 9}",65,M,53000.0
73,offer completed,9,498,"{'reward': 2, 'offer id': 10}",65,M,53000.0
74,offer completed,9,498,"{'reward': 5, 'offer id': 4}",65,M,53000.0
80,offer completed,9,600,"{'reward': 5, 'offer id': 9}",65,M,53000.0
81,offer completed,9,600,"{'reward': 2, 'offer id': 7}",65,M,53000.0


In [42]:
offer_id_completed = []
reward = []
for val in df_completed['value']:
    offer_id_completed.append(val['offer id'])
    reward.append(val['reward'])

In [44]:
df_completed['offer_id_completed'] = offer_id_completed
df_completed['reward'] = reward

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [46]:
del df_completed['value']

In [47]:
df_completed

,event,user_id,time,age,gender,income,offer_id_completed,reward
3,offer completed,4,132,75,F,100000.0,4,5
14,offer completed,4,510,75,F,100000.0,1,10
15,offer completed,4,510,75,F,100000.0,9,5
39,offer completed,6,522,68,M,70000.0,4,5
40,offer completed,6,522,68,M,70000.0,7,2
65,offer completed,9,60,65,M,53000.0,9,5
73,offer completed,9,498,65,M,53000.0,10,2
74,offer completed,9,498,65,M,53000.0,4,5
80,offer completed,9,600,65,M,53000.0,9,5
81,offer completed,9,600,65,M,53000.0,7,2


In [50]:
df_viewed.rename(columns={"time":"time_viewed"}, inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [51]:
df_completed.rename(columns={"time":"time_completed"}, inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [52]:
df_viewed

,event,user_id,time_viewed,age,gender,income,offer_id_viewed
1,offer viewed,4,6,75,F,100000.0,4
6,offer viewed,4,216,75,F,100000.0,8
11,offer viewed,4,408,75,F,100000.0,1
17,offer viewed,4,582,75,F,100000.0,9
19,offer viewed,5,6,118,None,NaN,5
23,offer viewed,5,336,118,None,NaN,3
28,offer viewed,5,624,118,None,NaN,5
30,offer viewed,6,18,68,M,70000.0,10
35,offer viewed,6,420,68,M,70000.0,4
37,offer viewed,6,522,68,M,70000.0,7


In [54]:
del df_viewed['age']
del df_viewed['gender']
del df_viewed['income']

In [56]:
df_viewed_copy = df_viewed.copy()

In [59]:
df_viewed_copy.rename(columns={'offer_id_viewed': 'offer_id_completed'}, inplace=True)

In [61]:
# This dataframe is only for individuals who viewed and completed the offer.

df_viewed_completed = df_viewed_copy.merge(df_completed, on=['user_id', 'offer_id_completed'])

In [62]:
df_viewed_completed

,event_x,user_id,time_viewed,offer_id_completed,event_y,time_completed,age,gender,income,reward
0,offer viewed,4,6,4,offer completed,132,75,F,100000.0,5
1,offer viewed,4,408,1,offer completed,510,75,F,100000.0,10
2,offer viewed,4,582,9,offer completed,510,75,F,100000.0,5
3,offer viewed,6,420,4,offer completed,522,68,M,70000.0,5
4,offer viewed,6,522,7,offer completed,522,68,M,70000.0,2
5,offer viewed,9,0,9,offer completed,60,65,M,53000.0,5
6,offer viewed,9,0,9,offer completed,600,65,M,53000.0,5
7,offer viewed,9,504,9,offer completed,60,65,M,53000.0,5
8,offer viewed,9,504,9,offer completed,600,65,M,53000.0,5
9,offer viewed,9,192,4,offer completed,498,65,M,53000.0,5


In [69]:
df_viewed_completed[df_viewed_completed['age'] > 95]

,event_x,user_id,time_viewed,offer_id_completed,event_y,time_completed,age,gender,income,reward
13,offer viewed,11,684,1,offer completed,684,118,None,NaN,10
46,offer viewed,27,414,7,offer completed,498,118,None,NaN,2
55,offer viewed,34,534,9,offer completed,612,96,F,89000.0,5
63,offer viewed,37,504,9,offer completed,594,118,None,NaN,5
66,offer viewed,40,378,9,offer completed,426,118,None,NaN,5
75,offer viewed,46,24,7,offer completed,96,118,None,NaN,2
193,offer viewed,97,0,7,offer completed,216,118,None,NaN,2
391,offer viewed,1305,336,6,offer completed,432,118,None,NaN,3
395,offer viewed,222,408,4,offer completed,558,118,None,NaN,5
466,offer viewed,254,354,6,offer completed,396,118,None,NaN,3
